# 1. Parameters

In [1]:
# Defaults

## Random seed
random_seed <- 25524

## Directories
simulation_dir <- "simulations/unset"
reference_file <- "simulations/reference/reference.fa.gz"
initial_tree_file <- "input/salmonella.tre"

## Simulation parameters
sub_lambda <- 1e-2
sub_pi_tcag <- c(0.1, 0.2, 0.3, 0.4)
sub_alpha <- 0.2
sub_beta <- 0.1
sub_mu <- 1
sub_invariant <- 0.3
ins_rate <- 1e-4
ins_max_length <- 60
ins_a <- 1.6
del_rate <- 1e-4
del_max_length <- 60
del_a <- 1.6

## Read simulation information
read_coverage <- 30
read_length <- 250

## Other
ncores <- 48

In [2]:
output_dir <- file.path(simulation_dir, "simulated_data")
output_vcf_prefix <- file.path(output_dir, "haplotypes")
reads_data_initial_prefix <- file.path(output_dir, "reads_initial", "data")
set.seed(random_seed)
print(output_dir)
print(output_vcf_prefix)

[1] "simulations/unset/simulated_data"
[1] "simulations/unset/simulated_data/haplotypes"


# 2. Generate simulated data

This simulates *Salmonella* data using a reference genome and a tree.

In [3]:
library(jackalope)

# Make sure we've complied with openmp
jackalope:::using_openmp()

[1] TRUE

In [4]:
reference <- read_fasta(reference_file)
reference_len <- reference$sizes()
reference

< Set of 1 chromosomes >
# Total size: 10,834 bp
  name                             chromosome                             length
chrom0     GTATTGCGAGGGTGGGGGAGTACGGCAG...CAAGAATGCAACCGACTCTGTCGGGACT     10834

In [5]:
library(ape)

tree <- read.tree(initial_tree_file)
tree <- root(tree, "reference", resolve.root=TRUE)
tree


Phylogenetic tree with 60 tips and 59 internal nodes.

Tip labels:
  reference, SH12-013, SH12-014, SH10-015, SH10-014, SH11-002, ...
Node labels:
  Root, 1.000000, 0.000000, 0.000000, 1.000000, 0.000000, ...

Rooted; includes branch lengths.

In [6]:
sub <- sub_HKY85(pi_tcag = sub_pi_tcag, mu = sub_mu,
                 alpha = sub_alpha, beta = sub_beta, gamma_k = 5,
                 invariant = sub_invariant)
ins <- indels(rate = ins_rate, max_length = ins_max_length, a = ins_a)
del <- indels(rate = del_rate, max_length = del_max_length, a = del_a)

ref_haplotypes <- create_haplotypes(reference, haps_phylo(tree), sub=sub, ins=ins, del=del)
ref_haplotypes

                              << haplotypes object >>
# Haplotypes: 60
# Mutations: 10,109

                          << Reference genome info: >>
< Set of 1 chromosomes >
# Total size: 10,834 bp
  name                             chromosome                             length
chrom0     GTATTGCGAGGGTGGGGGAGTACGGCAG...CAAGAATGCAACCGACTCTGTCGGGACT     10834

# 3. Write simulated data

In [7]:
write_vcf(ref_haplotypes, out_prefix=output_vcf_prefix, compress=TRUE)

In [8]:
assemblies_prefix = file.path(output_dir, "assemblies", "data")

write_fasta(ref_haplotypes, out_prefix=assemblies_prefix,
            compress=TRUE, n_threads=ncores, overwrite=TRUE)

In [9]:
n_samples <- length(tree$tip)
n_reads <- round((reference_len * read_coverage * n_samples) / read_length)
print(sprintf("Number of reads for coverage %sX and read length %s over %s samples with respect to reference with length %s: %s", 
              read_coverage, read_length, n_samples, reference_len, n_reads))

illumina(ref_haplotypes, out_prefix = reads_data_initial_prefix, sep_files=TRUE, n_reads = n_reads,
         frag_mean = read_length * 2 + 50, frag_sd = 100,
         compress=TRUE, comp_method="bgzip", n_threads=ncores,
         paired=TRUE, read_length = read_length)

[1] "Number of reads for coverage 30X and read length 250 over 60 samples with respect to reference with length 10834: 78005"


In [10]:
# Remove the simulated reads for the reference genome since I don't want these in the tree
ref1 <- paste(toString(reads_data_initial_prefix), "_reference_R1.fq.gz", sep="")
ref2 <- paste(toString(reads_data_initial_prefix), "_reference_R2.fq.gz", sep="")
if (file.exists(ref1)) {
    file.remove(ref1)
    print(sprintf("Removing: %s", ref1))
}
if (file.exists(ref2)) {
    file.remove(ref2)
    print(sprintf("Removing: %s", ref2))
}

[1] "Removing: simulations/unset/simulated_data/reads_initial/data_reference_R1.fq.gz"
[1] "Removing: simulations/unset/simulated_data/reads_initial/data_reference_R2.fq.gz"


In [11]:
# Remove the new reference assembly genome since I don't need it
ref1 <- paste(toString(assemblies_prefix), "__reference.fa.gz", sep="")
if (file.exists(ref1)) {
    file.remove(ref1)
    print(sprintf("Removing: %s", ref1))
}

[1] "Removing: simulations/unset/simulated_data/assemblies/data__reference.fa.gz"
